In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers 
import torch

## Polbert - Polish BERT
Polish version of BERT language model is here! It is now available in two variants: cased and uncased, both can be downloaded and used via HuggingFace transformers library. I recommend using the cased model, more info on the differences and benchmark results below.



In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, pipeline
model_bert = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")
tokenizer_bert = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def bert_predict(sentance):
    nlp = pipeline('fill-mask', model=model_bert, tokenizer=tokenizer_bert)
    first_part = sentance.split("<mask>")[0]
    second_part = sentance.split("<mask>")[1]
    merged_sentance = f"{first_part} {nlp.tokenizer.mask_token} {second_part}"
    result = nlp(merged_sentance)

    for i, r in enumerate(result):
        print(f"Predicions {i}: score={r['score']:.2f} -> {r['sequence']}")


In [ ]:
bert_predict("Prezydent <mask> wygrał z Trumpem")

Predicions 0: score=0.31 -> Prezydent Obama wygrał z Trumpem
Predicions 1: score=0.25 -> Prezydent Trump wygrał z Trumpem
Predicions 2: score=0.07 -> Prezydent USA wygrał z Trumpem
Predicions 3: score=0.04 -> Prezydent Clinton wygrał z Trumpem
Predicions 4: score=0.04 -> Prezydent Bush wygrał z Trumpem


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer_roberta = AutoTokenizer.from_pretrained('xlm-roberta-large')
model_roberta = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")


### XLM-RoBERTa (large-sized model)

RoBERTa is a transformers model pretrained on a large corpus in a self-supervised fashion. This means it was pretrained on the raw texts only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts.

In [ ]:
def roberta_predict(sentance):
    nlp = pipeline('fill-mask', model="xlm-roberta-large")
    result = nlp(sentance)

    for i, r in enumerate(result):
        print(f"Predicions {i}: score={r['score']:.2f} -> {r['sequence']}")


In [ ]:
roberta_predict("Prezydent <mask> wygrał z Trumpem")

Predicions 0: score=0.17 -> Prezydent Peru wygrał z Trumpem
Predicions 1: score=0.17 -> Prezydent Putin wygrał z Trumpem
Predicions 2: score=0.11 -> Prezydent Obama wygrał z Trumpem
Predicions 3: score=0.04 -> Prezydent Macron wygrał z Trumpem
Predicions 4: score=0.04 -> Prezydent USA wygrał z Trumpem


### TwHIN-BERT: A Socially-Enriched Pre-trained Language Model for Multilingual Tweet Representations
TwHIN-BERT is a new multi-lingual Tweet language model that is trained on 7 billion Tweets from over 100 distinct languages. TwHIN-BERT differs from prior pre-trained language models as it is trained with not only text-based self-supervision (e.g., MLM), but also with a social objective based on the rich social engagements within a Twitter Heterogeneous Information Network (TwHIN).

TwHIN-BERT can be used as a drop-in replacement for BERT in a variety of NLP and recommendation tasks. It not only outperforms similar models semantic understanding tasks such text classification), but also social recommendation tasks such as predicting user to Tweet engagement.

In [ ]:
def twhin_bert(sentance):
    nlp = pipeline('fill-mask', model='Twitter/twhin-bert-base')
    result = nlp(sentance)
    for i, r in enumerate(result):
        print(f"Predicions {i}: score={r['score']:.2f} -> {r['sequence']}")


In [ ]:
twhin_bert("Prezydent <mask> wygrał z Trumpem")

Predicions 0: score=0.40 -> Prezydent USA wygrał z Trumpem
Predicions 1: score=0.19 -> Prezydent Warszawy wygrał z Trumpem
Predicions 2: score=0.12 -> Prezydent Polski wygrał z Trumpem
Predicions 3: score=0.03 -> Prezydent RP wygrał z Trumpem
Predicions 4: score=0.03 -> Prezydent Trump wygrał z Trumpem


## Devise a method to test if the langage model understands Polish cases. E.g. testing for nominal case could be expressed as "Warszawa to największe [MASK]", and the masked word should be in nominative case. Create sentences for each case.

In [ ]:
# mianownik:
print("bert_predict")
bert_predict("Warszawa to największe <mask>")

print("roberta_predict")
roberta_predict("Warszawa to największe <mask>")

print("twhin_bert")
twhin_bert("Warszawa to największe <mask>")


bert_predict
Predicions 0: score=0.92 -> Warszawa to największe miasto
Predicions 1: score=0.01 -> Warszawa to największe.
Predicions 2: score=0.01 -> Warszawa to największe miejsce
Predicions 3: score=0.01 -> Warszawa to największe województwo
Predicions 4: score=0.01 -> Warszawa to największe lotnisko
roberta_predict
Predicions 0: score=0.95 -> Warszawa to największe miasto
Predicions 1: score=0.03 -> Warszawa to największe miasta
Predicions 2: score=0.01 -> Warszawa to największe centrum
Predicions 3: score=0.00 -> Warszawa to największe Miasto
Predicions 4: score=0.00 -> Warszawa to największe...
twhin_bert
Predicions 0: score=0.09 -> Warszawa to największe miasto
Predicions 1: score=0.07 -> Warszawa to największe święto
Predicions 2: score=0.06 -> Warszawa to największe szczęście
Predicions 3: score=0.06 -> Warszawa to największe życie
Predicions 4: score=0.04 -> Warszawa to największe miejsce


In [ ]:
# dopełniacz:
print("bert_predict")
bert_predict("Mój dom jest naprzeciwko <mask>, więc chodzimy tam robić zakupy")

print("roberta_predict")
roberta_predict("Mój dom jest naprzeciwko <mask>, więc chodzimy tam robić zakupy")

print("twhin_bert")
twhin_bert("Mój dom jest naprzeciwko <mask>, więc chodzimy tam robić zakupy")

bert_predict
Predicions 0: score=0.09 -> Mój dom jest naprzeciwko mnie, więc chodzimy tam robić zakupy
Predicions 1: score=0.06 -> Mój dom jest naprzeciwko domu, więc chodzimy tam robić zakupy
Predicions 2: score=0.05 -> Mój dom jest naprzeciwko szkoły, więc chodzimy tam robić zakupy
Predicions 3: score=0.03 -> Mój dom jest naprzeciwko kościoła, więc chodzimy tam robić zakupy
Predicions 4: score=0.03 -> Mój dom jest naprzeciwko plaży, więc chodzimy tam robić zakupy
roberta_predict
Predicions 0: score=0.66 -> Mój dom jest naprzeciwko sklepu, więc chodzimy tam robić zakupy
Predicions 1: score=0.14 -> Mój dom jest naprzeciwko Tesco, więc chodzimy tam robić zakupy
Predicions 2: score=0.03 -> Mój dom jest naprzeciwko rynku, więc chodzimy tam robić zakupy
Predicions 3: score=0.02 -> Mój dom jest naprzeciwko Lidl, więc chodzimy tam robić zakupy
Predicions 4: score=0.01 -> Mój dom jest naprzeciwko szkoły, więc chodzimy tam robić zakupy
twhin_bert
Predicions 0: score=0.09 -> Mój dom jest naprze

In [ ]:
# celownik:
print("bert_predict")
bert_predict("Zapomniałem kupić mojej <mask> prezentu na Walentynki.")

print("roberta_predict")
roberta_predict("Zapomniałem kupić mojej <mask> prezentu na Walentynki.")

print("twhin_bert")
twhin_bert("Zapomniałem kupić mojej <mask> prezentu na Walentynki.")


bert_predict
Predicions 0: score=0.36 -> Zapomniałem kupić mojej mamie prezentu na Walentynki.
Predicions 1: score=0.19 -> Zapomniałem kupić mojej żonie prezentu na Walentynki.
Predicions 2: score=0.13 -> Zapomniałem kupić mojej dziewczynie prezentu na Walentynki.
Predicions 3: score=0.08 -> Zapomniałem kupić mojej matce prezentu na Walentynki.
Predicions 4: score=0.05 -> Zapomniałem kupić mojej córce prezentu na Walentynki.
roberta_predict
Predicions 0: score=0.33 -> Zapomniałem kupić mojej mami prezentu na Walentynki.
Predicions 1: score=0.28 -> Zapomniałem kupić mojej mamy prezentu na Walentynki.
Predicions 2: score=0.12 -> Zapomniałem kupić mojej Kasi prezentu na Walentynki.
Predicions 3: score=0.05 -> Zapomniałem kupić mojej Mamy prezentu na Walentynki.
Predicions 4: score=0.04 -> Zapomniałem kupić mojej pani prezentu na Walentynki.
twhin_bert
Predicions 0: score=0.86 -> Zapomniałem kupić mojej mamy prezentu na Walentynki.
Predicions 1: score=0.02 -> Zapomniałem kupić mojej pani p

In [ ]:
# biernik:
print("bert_predict")
bert_predict("Lubię czarną <mask>.")

print("roberta_predict")
roberta_predict("Lubię czarną <mask>.")
print("twhin_bert")
twhin_bert("Lubię czarną <mask>.")


bert_predict
Predicions 0: score=0.11 -> Lubię czarną muzykę.
Predicions 1: score=0.10 -> Lubię czarną kawę.
Predicions 2: score=0.07 -> Lubię czarną skórę.
Predicions 3: score=0.05 -> Lubię czarną magię.
Predicions 4: score=0.04 -> Lubię czarną kobietę.
roberta_predict
Predicions 0: score=0.32 -> Lubię czarną skórę.
Predicions 1: score=0.03 -> Lubię czarną stronę.
Predicions 2: score=0.03 -> Lubię czarną noc.
Predicions 3: score=0.03 -> Lubię czarną miłość.
Predicions 4: score=0.03 -> Lubię czarną wodę.
twhin_bert
Predicions 0: score=0.16 -> Lubię czarną kąpiel.
Predicions 1: score=0.11 -> Lubię czarną wodę.
Predicions 2: score=0.10 -> Lubię czarną twarz.
Predicions 3: score=0.06 -> Lubię czarną biały.
Predicions 4: score=0.04 -> Lubię czarną osobę.


In [ ]:
# Narzędnik:
print("bert_predict")
bert_predict("Interesuję się <mask>, więc czytam wiele książek")

print("roberta_predict")
roberta_predict("Interesuję się <mask>, więc czytam wiele książek")
print("twhin_bert")
twhin_bert("Interesuję się <mask>, więc czytam wiele książek")


bert_predict
Predicions 0: score=0.13 -> Interesuję się sztuką, więc czytam wiele książek
Predicions 1: score=0.07 -> Interesuję się muzyką, więc czytam wiele książek
Predicions 2: score=0.05 -> Interesuję się nauką, więc czytam wiele książek
Predicions 3: score=0.05 -> Interesuję się tym, więc czytam wiele książek
Predicions 4: score=0.05 -> Interesuję się historią, więc czytam wiele książek
roberta_predict
Predicions 0: score=0.14 -> Interesuję się fantasy, więc czytam wiele książek
Predicions 1: score=0.14 -> Interesuję się wszystkim, więc czytam wiele książek
Predicions 2: score=0.07 -> Interesuję się wiedzą, więc czytam wiele książek
Predicions 3: score=0.06 -> Interesuję się literatura, więc czytam wiele książek
Predicions 4: score=0.05 -> Interesuję się dziećmi, więc czytam wiele książek
twhin_bert
Predicions 0: score=0.16 -> Interesuję się książek, więc czytam wiele książek
Predicions 1: score=0.13 -> Interesuję się książki, więc czytam wiele książek
Predicions 2: score=0.09 ->

In [ ]:
# miejscownik:
print("bert_predict")
bert_predict("O mojej <mask> wolałbym nie rozmawiać.")

print("roberta_predict")
roberta_predict("O mojej <mask> wolałbym nie rozmawiać.")

print("twhin_bert")
twhin_bert("O mojej <mask> wolałbym nie rozmawiać.")


bert_predict
Predicions 0: score=0.14 -> O mojej żonie wolałbym nie rozmawiać.
Predicions 1: score=0.10 -> O mojej pracy wolałbym nie rozmawiać.
Predicions 2: score=0.09 -> O mojej rodzinie wolałbym nie rozmawiać.
Predicions 3: score=0.06 -> O mojej przyszłości wolałbym nie rozmawiać.
Predicions 4: score=0.06 -> O mojej śmierci wolałbym nie rozmawiać.
roberta_predict
Predicions 0: score=0.16 -> O mojej pracy wolałbym nie rozmawiać.
Predicions 1: score=0.11 -> O mojej przyszłości wolałbym nie rozmawiać.
Predicions 2: score=0.09 -> O mojej przeszłości wolałbym nie rozmawiać.
Predicions 3: score=0.07 -> O mojej sytuacji wolałbym nie rozmawiać.
Predicions 4: score=0.07 -> O mojej opinii wolałbym nie rozmawiać.
twhin_bert
Predicions 0: score=0.16 -> O mojej głowie wolałbym nie rozmawiać.
Predicions 1: score=0.09 -> O mojej pracy wolałbym nie rozmawiać.
Predicions 2: score=0.09 -> O mojej przyszłości wolałbym nie rozmawiać.
Predicions 3: score=0.04 -> O mojej szkole wolałbym nie rozmawiać.
P

In [ ]:
# wolacz:
print("bert_predict")
bert_predict("Szanowna Pani <mask> !")

print("roberta_predict")
roberta_predict("Szanowna Pani <mask> !")


print("twhin_bert")
twhin_bert("Szanowna Pani <mask> !")



bert_predict
Predicions 0: score=0.81 -> Szanowna Pani Marszałek!
Predicions 1: score=0.07 -> Szanowna Pani Minister!
Predicions 2: score=0.07 -> Szanowna Pani Poseł!
Predicions 3: score=0.02 -> Szanowna Pani Premier!
Predicions 4: score=0.01 -> Szanowna Pani Prezes!
roberta_predict
Predicions 0: score=0.22 -> Szanowna Panie!
Predicions 1: score=0.18 -> Szanowna Pani Dyrektor!
Predicions 2: score=0.17 -> Szanowna Pani Prezes!
Predicions 3: score=0.07 -> Szanowna Pani Minister!
Predicions 4: score=0.03 -> Szanowna Pani Doktor!
twhin_bert
Predicions 0: score=0.70 -> Szanowna Panie!
Predicions 1: score=0.03 -> Szanowna Pani Merkel!
Predicions 2: score=0.02 -> Szanowna Panią!
Predicions 3: score=0.02 -> Szanowna Pani Ukraina!
Predicions 4: score=0.01 -> Szanowna Pani Boże!


## Devise a method to test long-range relationships such as gender. E.e. you can use two verbs where withe masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

In [ ]:
# test:
sentance = "Dawno nie widziałem się z rodziami, mama <mask> na wakacje a tata pracuje. "
print("bert_predict")
bert_predict(sentance)

print("roberta_predict")
roberta_predict(sentance)


print("twhin_bert")
twhin_bert(sentance)



bert_predict
Predicions 0: score=0.32 -> Dawno nie widziałem się z rodziami, mama wyjechała na wakacje a tata pracuje.
Predicions 1: score=0.31 -> Dawno nie widziałem się z rodziami, mama wyjeżdża na wakacje a tata pracuje.
Predicions 2: score=0.12 -> Dawno nie widziałem się z rodziami, mama przyjeżdża na wakacje a tata pracuje.
Predicions 3: score=0.08 -> Dawno nie widziałem się z rodziami, mama jedzie na wakacje a tata pracuje.
Predicions 4: score=0.07 -> Dawno nie widziałem się z rodziami, mama pojechała na wakacje a tata pracuje.
roberta_predict
Predicions 0: score=0.41 -> Dawno nie widziałem się z rodziami, mama była na wakacje a tata pracuje.
Predicions 1: score=0.28 -> Dawno nie widziałem się z rodziami, mama jest na wakacje a tata pracuje.
Predicions 2: score=0.10 -> Dawno nie widziałem się z rodziami, mama idzie na wakacje a tata pracuje.
Predicions 3: score=0.06 -> Dawno nie widziałem się z rodziami, mama wychodzi na wakacje a tata pracuje.
Predicions 4: score=0.04 -> Dawno n

In [ ]:
# test:
sentance = "Umyłem  buty żony, bo <mask> je biegając. "
print("bert_predict")
bert_predict(sentance)

print("roberta_predict")
roberta_predict(sentance)


print("twhin_bert")
twhin_bert(sentance)



bert_predict
Predicions 0: score=0.09 -> Umyłem buty żony, bo nosiła je biegając.
Predicions 1: score=0.04 -> Umyłem buty żony, bo znalazłem je biegając.
Predicions 2: score=0.04 -> Umyłem buty żony, bo zgubiłem je biegając.
Predicions 3: score=0.04 -> Umyłem buty żony, bo miała je biegając.
Predicions 4: score=0.03 -> Umyłem buty żony, bo straciłem je biegając.
roberta_predict
Predicions 0: score=0.28 -> Umyłem buty żony, bo nosi je biegając.
Predicions 1: score=0.07 -> Umyłem buty żony, bo spala je biegając.
Predicions 2: score=0.06 -> Umyłem buty żony, bo pali je biegając.
Predicions 3: score=0.04 -> Umyłem buty żony, bo miała je biegając.
Predicions 4: score=0.04 -> Umyłem buty żony, bo zakłada je biegając.
twhin_bert
Predicions 0: score=0.10 -> Umyłem buty żony, bo lubię je biegając.
Predicions 1: score=0.09 -> Umyłem buty żony, bo mam je biegając.
Predicions 2: score=0.05 -> Umyłem buty żony, bo miałem je biegając.
Predicions 3: score=0.04 -> Umyłem buty żony, bo widzę je biegają

In [ ]:
# test:
sentance = "Syn chodził na zajęcia, mimo tego nauczycielka  nie <mask> go do kolejnej klasy. "
print("bert_predict")
bert_predict(sentance)

print("roberta_predict")
roberta_predict(sentance)


print("twhin_bert")
twhin_bert(sentance)



bert_predict
Predicions 0: score=0.17 -> Syn chodził na zajęcia, mimo tego nauczycielka nie chciała go do kolejnej klasy.
Predicions 1: score=0.12 -> Syn chodził na zajęcia, mimo tego nauczycielka nie zabrała go do kolejnej klasy.
Predicions 2: score=0.09 -> Syn chodził na zajęcia, mimo tego nauczycielka nie wysłała go do kolejnej klasy.
Predicions 3: score=0.06 -> Syn chodził na zajęcia, mimo tego nauczycielka nie oddała go do kolejnej klasy.
Predicions 4: score=0.05 -> Syn chodził na zajęcia, mimo tego nauczycielka nie przeniosła go do kolejnej klasy.
roberta_predict
Predicions 0: score=0.16 -> Syn chodził na zajęcia, mimo tego nauczycielka nie wprowadza go do kolejnej klasy.
Predicions 1: score=0.09 -> Syn chodził na zajęcia, mimo tego nauczycielka nie chciała go do kolejnej klasy.
Predicions 2: score=0.08 -> Syn chodził na zajęcia, mimo tego nauczycielka nie wysyła go do kolejnej klasy.
Predicions 3: score=0.05 -> Syn chodził na zajęcia, mimo tego nauczycielka nie oddaje go do kole

## Check if the model captures real-world knolwedge. For instance a sentence "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [ ]:
sentance = "<mask> wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza"

print("bert_predict")
bert_predict(sentance)

print("roberta_predict")
roberta_predict(sentance)


print("twhin_bert")
twhin_bert(sentance)



bert_predict
Predicions 0: score=0.21 -> Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza
Predicions 1: score=0.07 -> Słońce wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza
Predicions 2: score=0.06 -> Nie wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza
Predicions 3: score=0.06 -> Ziemia wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza
Predicions 4: score=0.05 -> Piwo wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza
roberta_predict
Predicions 0: score=0.06 -> Olej wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza
Predicions 1: score=0.05 -> Sól wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza
Predicions 2: score=0.03 -> śnieg wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza
Predicions 3: score=0.03 -> Alkohol wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 s

In [ ]:
sentance = "Satelitą Ziemi jest <mask>"

print("bert_predict")
bert_predict(sentance)

print("roberta_predict")
roberta_predict(sentance)


print("twhin_bert")
twhin_bert(sentance)



bert_predict
Predicions 0: score=0.48 -> Satelitą Ziemi jest :
Predicions 1: score=0.02 -> Satelitą Ziemi jest.
Predicions 2: score=0.02 -> Satelitą Ziemi jest Ziemia
Predicions 3: score=0.02 -> Satelitą Ziemi jest satelita
Predicions 4: score=0.01 -> Satelitą Ziemi jest Księżyc
roberta_predict
Predicions 0: score=0.44 -> Satelitą Ziemi jest...
Predicions 1: score=0.34 -> Satelitą Ziemi jest:
Predicions 2: score=0.04 -> Satelitą Ziemi jest...
Predicions 3: score=0.02 -> Satelitą Ziemi jest.
Predicions 4: score=0.01 -> Satelitą Ziemi jest NASA
twhin_bert
Predicions 0: score=0.09 -> Satelitą Ziemi jest.
Predicions 1: score=0.06 -> Satelitą Ziemi jest Polska
Predicions 2: score=0.04 -> Satelitą Ziemi jest Putin
Predicions 3: score=0.03 -> Satelitą Ziemi jest...
Predicions 4: score=0.02 -> Satelitą Ziemi jest Bóg


In [ ]:
sentance = "Chrzest Polski był w <mask> roku"

print("bert_predict")
bert_predict(sentance)

print("roberta_predict")
roberta_predict(sentance)


print("twhin_bert")
twhin_bert(sentance)



bert_predict
Predicions 0: score=0.03 -> Chrzest Polski był w 1921 roku
Predicions 1: score=0.03 -> Chrzest Polski był w 1945 roku
Predicions 2: score=0.02 -> Chrzest Polski był w 1946 roku
Predicions 3: score=0.02 -> Chrzest Polski był w 1918 roku
Predicions 4: score=0.02 -> Chrzest Polski był w 1939 roku
roberta_predict
Predicions 0: score=0.11 -> Chrzest Polski był w 1918 roku
Predicions 1: score=0.06 -> Chrzest Polski był w 1939 roku
Predicions 2: score=0.03 -> Chrzest Polski był w 1989 roku
Predicions 3: score=0.02 -> Chrzest Polski był w 1914 roku
Predicions 4: score=0.02 -> Chrzest Polski był w 1991 roku
twhin_bert
Predicions 0: score=0.09 -> Chrzest Polski był w 2015 roku
Predicions 1: score=0.06 -> Chrzest Polski był w 2008 roku
Predicions 2: score=0.05 -> Chrzest Polski był w 2014 roku
Predicions 3: score=0.05 -> Chrzest Polski był w tym roku
Predicions 4: score=0.05 -> Chrzest Polski był w 2019 roku


In [ ]:
sentance = "Jeżeli zamrozisz wodę to zamienia się w  <mask>"

print("bert_predict")
bert_predict(sentance)

print("roberta_predict")
roberta_predict(sentance)


print("twhin_bert")
twhin_bert(sentance)



bert_predict
Predicions 0: score=0.09 -> Jeżeli zamrozisz wodę to zamienia się w wodę
Predicions 1: score=0.07 -> Jeżeli zamrozisz wodę to zamienia się w kamień
Predicions 2: score=0.04 -> Jeżeli zamrozisz wodę to zamienia się w lód
Predicions 3: score=0.04 -> Jeżeli zamrozisz wodę to zamienia się w krew
Predicions 4: score=0.02 -> Jeżeli zamrozisz wodę to zamienia się w wino
roberta_predict
Predicions 0: score=0.14 -> Jeżeli zamrozisz wodę to zamienia się w alkohol
Predicions 1: score=0.08 -> Jeżeli zamrozisz wodę to zamienia się w gaz
Predicions 2: score=0.05 -> Jeżeli zamrozisz wodę to zamienia się w wodę
Predicions 3: score=0.04 -> Jeżeli zamrozisz wodę to zamienia się w tłuszcz
Predicions 4: score=0.04 -> Jeżeli zamrozisz wodę to zamienia się w cukier
twhin_bert
Predicions 0: score=0.06 -> Jeżeli zamrozisz wodę to zamienia się w wodę
Predicions 1: score=0.03 -> Jeżeli zamrozisz wodę to zamienia się w życie
Predicions 2: score=0.02 -> Jeżeli zamrozisz wodę to zamienia się w prąd
Pr

Answer the following questions:
Which of the models produced the best results?
- najlepsze wyniki zawsze daje Polish Bert

Was any of the models able to capture Polish grammar?
- wszystkie dobrze sobie radzily z gramatyka

Was any of the models able to capture long-distant relationships between the words?
- Polish-bert i Robert

Was any of the models able to capture world knowledge?
- jedyne opprawne sygestie dał Polish Bert ale równiez nie ma dobrey knowledge-base

What are the most striking errors made by the models?
- twhin_bert nie radził sobie z rozumieniem ogólnego kontekstu